In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import mplcursors
%matplotlib notebook
#%matplotlib widget
import pandas as pd
from matplotlib.lines import Line2D
from scipy.stats import linregress 
from scipy.fft import fft, rfft,fftshift
from scipy.fft import fftfreq, rfftfreq

from scipy.fft import fft, rfft,fftshift
from scipy.fft import fftfreq, rfftfreq
import sys
import utilities
import importlib
importlib.reload(utilities)
import json
import pickle

Reading an lc code:

In [ ]:
df=pd.read_csv("classified_lcs\grs1915_lc"+str(181)+".txt",sep="	",skiprows=[0, 1], header=None)
df.columns = ['time', 'total','low','mid','high'] #time in seconds

In [ ]:
len(df)

In [ ]:
df

In [ ]:
#loading the shaplet
with open("shaplet.pkl", "rb") as f:
    shaplet = pickle.load(f)

In [ ]:
%matplotlib inline
plt.title("The shaplet used for analysis")
plt.plot(shaplet)
plt.show()

Analysis:

In [ ]:
len(shaplet)

In [ ]:
with open("computed_data.pkl", "rb") as f:
    data = pickle.load(f)  # Assuming it's a dictionary

print(data.keys())

In [ ]:
min_distances_table=np.array(data['min_distances_table'])

In [ ]:
argsort_min=np.array(data["argsort_min"])

In [ ]:
argsort_min_=np.array(data["argsort_min_without_nan"])

In [ ]:
distancestable=data["dist_table"]

In [ ]:
summarylcindeces=np.array(data["first_column"]).astype(int)

In [ ]:
summarypositions=np.array(data["second_column"]).astype(int)

In [ ]:
summaryminimas=np.array(data["third_column"])

In [ ]:
summarytable=[]
for i in range(len(data["first_column"])):
    summarytable.append([data["first_column"][i],data["second_column"][i],data["third_column"][i]])

In [ ]:
#summary table :      light curve index , minimum position , chi^2      
#it sorted based on the minimas

In [ ]:
quality=[0]*len(distancestable) #i ll store the quality of the shaplet matching or not the signal
count=[0]*len(distancestable)

In [ ]:
cut=3

In [ ]:
n=np.sum(summaryminimas<cut)

for i in range(n):
    lc_index=summarylcindeces[i]
    count[lc_index]+=1

In [ ]:
count

In [ ]:
count[181]

In [ ]:
l=np.where(summarylcindeces==181)
c=0

In [ ]:
np.sum( (summarylcindeces==181)&(summaryminimas<cut))

In [ ]:
summarypositions[(summarylcindeces==181)&(summaryminimas<cut)]

In [ ]:
#normalisation for each lc index:
for i in range(len(count)):
    df=pd.read_csv("classified_lcs\grs1915_lc"+str(i)+".txt",sep="	",skiprows=[0, 1], header=None)
    quality[i]=count[i]*len(shaplet)/len(df)

In [ ]:
quality=np.array(quality)

In [ ]:
quality[quality!=0]

In [ ]:
plt.hist(quality[quality!=0])

In [ ]:
quality

In [ ]:
with open("classes.json", "r") as f:
    lcclasses = json.load(f)

In [ ]:
with open("key2class.json", "r") as f:
    key2class = json.load(f)

In [ ]:
classes_={}

In [ ]:
for i in range(len(count)):
    if(count[i]!=0):
        if (key2class[str(i)] in classes_.keys()):
            classes_[key2class[str(i)]].append(i)
        else:
            classes_[key2class[str(i)]]=[i]

In [ ]:
for key in classes_.keys():
    classes_[key]=np.array(classes_[key])

In [ ]:
classes_.keys()

In [ ]:
classes_=dict(sorted(classes_.items(), key=lambda item: len(item[1])))

In [ ]:
classes_

In [ ]:
len(lcclasses.keys())

In [ ]:
key2class["0"]

plot the curves where there are matches and see the matching

In [ ]:
#show a rho where there is no matching

In [ ]:
lcclasses["rho"]

Getting a rho that isnt in 

In [ ]:
lcclasses

In [ ]:
key="rho"
not_in={}
not_in[key]=[]
for el in lcclasses[key]:
    if(not (el in classes_[key])):
        not_in[key].append(el)

In [ ]:
classes_

In [ ]:
key="rho"

In [ ]:
%matplotlib inline
plt.figure()
for index in classes_[key]:
    low=utilities.get_low(index)
    legend="Class: "+key+"\nLC Number:"+str(index)+"\ndist minima: "+str(min_distances_table[index])
    utilities.draw(shaplet,[],low,txt=legend,minimasposition=summarypositions[(summarylcindeces==index)&(summaryminimas<5.2)])

In [ ]:
def drawwithcut(index,cut):
    #TO NOT FORGET: MATPLOTLIB WIDGET & PLT FIGURE!!!
    low=utilities.get_low(index)
    legend="Class: "+key+"\nLC Number:"+str(index)+"\ndist minima: "+str(min_distances_table[index])
    utilities.draw(shaplet,[],low,txt=legend,minimasposition=summarypositions[(summarylcindeces==index)&(summaryminimas<=cut)])
    

In [ ]:
%matplotlib widget
index=282
low=utilities.get_low(index)
legend="Class: "+key+"\nLC Number:"+str(index)+"\ndist minima: "+str(min_distances_table[index])
utilities.draw(shaplet,[],low,txt=legend,minimasposition=summarypositions[(summarylcindeces==index)&(summaryminimas<=5.2)])

In [ ]:
%matplotlib inline
for index in not_in[key]:
    low=utilities.get_low(index)
    legend="Class: "+key+"\nLC Number:"+str(index)+"\ndist minima: "+str(min_distances_table[index])
    utilities.draw(shaplet,distancestable[index],low,txt=legend)

In [ ]:
lcclasses

Rho signals analysis

In [ ]:
#defining the noholes function 

In [ ]:
def noholes(selectedpositions):
    if(len(selectedpositions)==0):   #NEED TO THINK ABOUT THE CASE WHERE I HAVE ONLY ONE ELEMENT
        return False
        
    condition=1
    selectedpositions=sorted(selectedpositions)
    for i in range(len(selectedpositions)-1):
        position= selectedpositions[i]
        nextposition=selectedpositions[i+1]
        if(abs((position+len(shaplet)-nextposition))/len(shaplet) > condition):
            print(position/8," ",nextposition/8)
       #     print(abs((position-nextposition))/len(shaplet))
            return False
    return True

In [ ]:
rhomaxcuts={}
rhomincuts={}

In [ ]:
summaryminimas[summarylcindeces==181]

In [ ]:
#get positions of after doing a cut:

In [ ]:
for el in lcclasses["rho"]:
    try:
        rhomaxcuts[el]=max(summaryminimas[summarylcindeces==el])
        rhomincuts[el]=min(summaryminimas[summarylcindeces==el])
    except:
        print(el)

In [ ]:
def positionsforcut(lcindex,cut):
    return summarypositions[(summarylcindeces==lcindex)&(summaryminimas<=cut)]

In [ ]:
#find minimum cut between max and min for no holes

In [ ]:
def getpositionsforcut(index,cut):
    return np.array(sorted(positionsforcut(index,cut)))

In [ ]:
noholes(getpositionsforcut(index,37))

In [ ]:
noholes(positionsforcut(500,rhomaxcuts[500]))

In [ ]:
def findcut(index,precision=1e-2):
    low=rhomincuts[index]
    high=rhomaxcuts[index]
    while high - low > precision:
        print(low,end=" ")
        print(high)
        mid = (low + high) / 2
        if noholes(positionsforcut(index,mid)):  
            high = mid  # Move left if `foo(mid)` is True
        else:
            low = mid   # Move right if `foo(mid)` is False
    return high

In [ ]:
findcut(500)

In [ ]:
chi_2=[]

In [ ]:
#finding the numebr of times shaplet was found in a curve
print(np.sum(summarylcindeces==181))
#finding the number of chi2 cut equivalent shaplet in a signal
max(summaryminimas[(summarylcindeces==181).nonzero()[0]])

In [ ]:
rhoefficientcut={}

In [ ]:
for index in lcclasses["rho"]:
    if(not noholes(getpositionsforcut(index,rhomaxcuts[index]))):
        rhoefficientcut[index]=-1
        continue
    rhoefficientcut[index]=findcut(index)

In [ ]:
rhochi2data = pd.DataFrame({'minimum cut': rhomincuts, 'efficient cut': rhoefficientcut,'max cut':rhomaxcuts})

In [ ]:
rhochi2data

In [ ]:
%matplotlib inline
plt.figure()
for ind in lcclasses["rho"]:
    if(rhoefficientcut[ind]==-1):
        continue
    drawwithcut(ind,rhoefficientcut[ind])

In [ ]:
np.sort(summarypositions[np.where(summarylcindeces==412)])

In [ ]:
%matplotlib widget
index=279
plt.figure()
plt.plot(distancestable[index])
plt.scatter(summarypositions[np.where(summarylcindeces==index)],distancestable[index][summarypositions[np.where(summarylcindeces==index)]])
plt.show()

In [ ]:
%matplotlib widget
index=279
plt.figure()
drawwithcut(index,findcut(index,5))

In [ ]:
findcut(279)

In [ ]:
getpositionsforcut(279,4.567)

##### SAVE THE PLOTS AND IMPORTANT FIGURES TO WHY WE CHOOSE A CERTAIN CUT 

In [ ]:
findcut(index,2)

In [ ]:
numberofshaplets={}

In [ ]:
qualityofshaplet={}

In [ ]:
getpositionsforcut(181,rhoefficientcut[181])

In [ ]:
for el in lcclasses["rho"]:
    if(rhomaxcuts[el]<=9):
        numberofshaplets[el]=len(getpositionsforcut(el,rhomaxcuts[el]))
        qualityofshaplet[el]=numberofshaplets[el]/(len(utilities.get_low(el))/len(shaplet))

In [ ]:
qualityofshaplet

In [ ]:
qualityshapletwithefficient={181: 0.8335302330576737,
 182: 0.9373107547381405,
 183: 1.004261522933722,
 184: 0.2556745182012848,
 185: 0.699000270197244,
 272: 0.7963792282039065,
 279: 0.6588365126911555,
 280: 0.0,
 283: 0.0,
 284: 0.0,
 287: 0.0,
 288: 0.0,
 299: 0.0,
 401: 0.0,
 411: 0.0,
 412: 0.0,
 413: 0.0,
 435: 0.0,
 439: 0.0,
 499: 0.0}


In [ ]:
plt.figure()
plt.hist(qualityofshaplet.values())

In [ ]:
summarypositions[0]

In [ ]:
with open("computed_data.pkl", "rb") as f:
    x = pickle.load(f)  # Assuming it's a dictionary

print(x.keys())

In [ ]:
summarypositionsx=np.array(x["second_column"]).astype(int)

In [ ]:
summarypositionsx[0]

In [ ]:
plt.figure()
plt.title("The shaplet used for analysis")
plt.xlabel("time in s")
plt.ylabel("intensity count /s ")
plt.plot(np.array(range(len(shaplet)))*0.125,shaplet,color='r')
plt.show()

In [ ]:
%matplotlib inline

df=pd.read_csv("classified_lcs\grs1915_lc"+str(181)+".txt",sep="	",skiprows=[0, 1], header=None)
df.columns = ["time", 'total','low','mid','high'] #time in seconds
low=np.array(list(df["low"]))

plt.figure()
plt.title("light curve 181,class rho")
plt.xlabel("time in s")
plt.ylabel("intensity count /s ")
plt.plot(np.array(range(len(low)))*0.125,low)

# Plot the selected part in a different color (e.g., red)
plt.plot(np.array(range(294,294+len(shaplet)))/8,shaplet, color='r')
plt.show()


In [ ]:
%matplotlib inline
x_point=2932/8
y_point=distancestable[181][int(x_point*8)]
plt.figure()
plt.xlabel("time in 1/8 s")
plt.ylabel("chi^2 / len(shaplet)")
plt.title("chi^2 plot")
plt.plot(np.array(range(len(distancestable[181])))/8,distancestable[181])
plt.plot(np.array(range(len(distancestable[181])))/8,[1]*len(distancestable[181]),color="orange")

# Show the coordinate values near the point
plt.text(0 , 1.5, f"1", fontsize=13, color="orange")

plt.text(x_point , y_point-1, f"({x_point:0.2f}, {y_point:0.2f})", fontsize=10, color="red")

plt.show()

In [ ]:
summaryminimas[summarylcindeces==181]

In [ ]:
sorted(summarypositions[(summaryminimas-1.26028)<0.01])

In [ ]:
%matplotlib widget

importlib.reload(utilities)

df=pd.read_csv("classified_lcs\grs1915_lc"+str(181)+".txt",sep="	",skiprows=[0, 1], header=None)
df.columns = ["time", 'total','low','mid','high'] #time in seconds
low=np.array(list(df["low"]))
plt.figure()
    
utilities.draw(shaplet,distancestable[181],low,txt="lc 181 compared to shaplet")
plt.show()
#plt.clf()

In [ ]:
summarypositions[summaryminimas==0]

In [ ]:
np.mean(summaryminimas)

In [ ]:
plt.figure()
plt.xlabel("min(chi^2) / lc")
plt.ylabel("number of lcs")
plt.title("histogramme of min(chi^2) per lc")
plt.hist(min_distances_table[argsort_min_],bins=100)
plt.show()

In [ ]:
plt.figure()
plt.xlabel("min(chi^2) / lc")
plt.ylabel("number of lcs")
plt.title("histogramme of min(chi^2) <= 10 per lc")
plt.hist(min_distances_table[min_distances_table<=10],bins=3)
plt.show()

In [ ]:
plt.figure()
plt.plot(distancestable[280]/)
plt.show()

In [ ]:
%matplotlib inline
plt.figure()
for el in lcclasses["rho"]:
    low=utilities.get_low(el)
    fourier = fftshift(fft(low))
    freqs = np.fft.fftshift(np.fft.fftfreq(len(low)))  # Normalized frequencies
   # plt.figure()
    # Plot the magnitude spectrum
    print(sorted(np.abs(fourier))[0])
    """
    plt.plot(freqs, np.abs(fourier))
    plt.xlim(-0.01,0.01)
    plt.xlabel("Frequency")
    plt.ylabel("Magnitude")
    plt.title("FFT result for the low component of the light curve 181,class rho")
    plt.show()"""